# Infarct Image Evaluation Notebook

## 📌 Overview
**InfarctImage** is a LoRA-based model fine-tuned on **Stable Diffusion 2.1** to generate realistic images of individuals simulating a heart attack. This model was developed to facilitate synthetic dataset generation for human activity recognition and medical emergency monitoring applications.

This notebook performs evaluations on models using **LPIPS**, comparing *Stable Diffusion 1.5 and 2.1* with their respective LoRA-tuned counterparts.

## 🎯 LPIPS
LPIPS (Learned Perceptual Image Patch Similarity) is an image quality metric that assesses the perceptual similarity between two images from a human perspective. Unlike traditional metrics such as MSE (Mean Squared Error) or PSNR (Peak Signal-to-Noise Ratio), which focus on pixel-by-pixel mathematical differences, LPIPS uses pre-trained neural networks to calculate perceptual distances. These networks extract high-level features from images and measure their similarity, allowing LPIPS to be more representative of how humans perceive differences between images.

In practical terms, LPIPS compares small fragments (patches) of the input images and generates a distance score. **Lower values ​​indicate that the images are more perceptually similar, while higher values ​​reflect greater differences**. This makes LPIPS ideal for assessing the quality of images generated by models such as Stable Diffusion, as it aligns better with human perception than traditional metrics.

## ✅ Evaluation
**Evaluation** was performed using:
```python
prompt = ("Elderly man at a sports stadium surrounded by a crowd, "
	"clutching his chest with a distressed look, indicating a heart attack."
	)

negative_prompt = (
	"blurry, deformed face, bad anatomy, poorly drawn face, out of focus, ugly, noisy, extra fingers, "
	"distorted, grainy, worst quality, low quality, low resolution, illustration, "
	"dull, watermark, close-up, 3d, 2d, painting, sketch, render, cartoon, grain, kitsch"
	)

trigger = "Person with expression of pain due to a heart attack, "

seeds = [
	1982408815, 2565723973, 20943342,   1098065577,  3359898783, 520125378,   581592661,   3782932107, 3010343430, 3482978058,  3039176121, 233994633,   2757580946,  549489462,   3415398733, 
	1068130776, 1083213219, 1089832411, 1149410080,  1159437037, 1208283828,  1238348364,  1246295977,  1304378311, 1308732453,  1318764911, 1346880011,  1348685546,  1358201195,  1383757460, 
	1384465663, 1395674011, 1406186486, 1416263893,  1464337940, 1481286912,  1487936185,  149839352,   1543054303, 1546229499,  1558024410, 1568821309,  1627952534,  1629369529,  1668527559,
	1730357672, 1751671856, 1799657764, 1820327791,  1847311362, 1851968161,  1869066466,  1921870256,  1960219230, 1962662508,  1968185403, 2003417982,  2113437653,  2114119715,  2125266420,
	2134568281, 2141966705, 2230643224, 2237772889,  2270525613, 2277882633,  2298753937,  2298822749,  232455271,  2329327261,  2329390460, 2346672635,  2354271158,  2376789774,  2412869895,
	2430449524, 2449180071, 2449262679, 2560166191,  2565782704, 2580801517,  2594236668,  2617410370,  2652820441, 2662133688,  2681600918, 2686545520,  2745868322,  2750932731,  2761328098,
	2775871759, 2788766922, 279564179,  2799573282,  2838338304, 2848342961,  2877344705,  2896484227,  2928698519, 2942326,     2991315415, 3038173830,  3057951538,  3074864865,  3141177100,
	3160956615, 3163851319, 3177839472, 317893606,   3189404692, 3269177408,  3294873161,  3297669937,  3318440864, 3318770946,  3341420384, 3344016972,  3373725245,  3396594899,  3409866889,
	3443794754, 345746834,  349805344,  350075436,   3517002786, 356429972,   35662756,    358946176,   3592771178, 3601261162,  3601381760, 3668629268,  3726569889,  376626356,   3781237347,
	3782607913, 3792531352, 3793384814, 3793586357,  3806828901, 3871703326,  3924417156,  3947344563,  3948153992, 3995364020,  40050947,   401889419,   4029264216,  4038072487,  4050538978
  ]
```

## 📘 use and contact

This notebook is part of a research or educational project.

👀 Ensure that you have the necessary dependencies installed before running the notebook.

To execute it successfully, you may need additional resources such as pretrained models or image datasets.

If you would like access to these resources, please contact me at:
lgabrielrojas@ucundinamarca.edu.co

License & Usage Notice
This notebook and its content are provided for academic and non-commercial purposes only, unless otherwise specified.
Redistribution or usage of the data or models must be done with proper attribution and in accordance with any included license files.

For collaborations, questions, or feedback, feel free to reach out via email.

## LPIPS Evaluation

In [1]:
#@title Only run this script if you have the necessary dependencies installed.
try:
	import numpy as np
except ImportError:
	print("numpy is not installed. Please install it using the command below:")
	print("Installing numpy...")
	! pip install numpy
	print("numpy installed successfully.")

try:
	import lpips
except ImportError:
	print("lpips is not installed. Please install it using the command below:")
	print("Installing lpips...")
	! pip install lpips
	print("lpips installed successfully.")

try:
	import pandas as pd
except ImportError:
	print("pandas is not installed. Please install it using the command below:")
	print("Installing pandas...")
	! pip install pandas
	print("pandas installed successfully.")

In [2]:
#@title Import necessary libraries
import lpips
import re
import torch
from PIL import Image
from torchvision import transforms
import os

In [3]:
#@title Pricipal variables
TEST_PATH = "test"
LPIPS_RESULTS = []

In [14]:
#@title Load LPIPS model and loss function
loss_fn = lpips.LPIPS(net='alex')  # 'alex', 'vgg', o 'squeeze' common choices
print("LPIPS model loaded successfully.")

# Images transform
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Ajusta al tamaño esperado
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalizar entre -1 y 1
])

# Function to calculate LPIPS distance between two images
def calculate_lpips(image1, image2):
		"""Calculate the LPIPS distance between two images.
		Args:
			image1 (str): Path to the first image or a PIL Image.
			image2 (str): Path to the second image or a PIL Image.
		Returns:
			float: The LPIPS distance between the two images.
		"""
		if isinstance(image1, str):
			img1 = Image.open(image1).convert("RGB")
		else:
			img1 = image1.convert("RGB")
		if isinstance(image2, str):
			img2 = Image.open(image2).convert("RGB")
		else:
			img2 = image2.convert("RGB")

		img1 = transform(img1).unsqueeze(0)  # Add batch dimension
		img2 = transform(img2).unsqueeze(0)
		# distance LPIPS
		with torch.no_grad():
				distance = loss_fn(img1, img2)
		return distance.item()

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


d:\Anaconda\anaconda3\envs\infarct\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda\anaconda3\envs\infarct\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: d:\Anaconda\anaconda3\envs\infarct\Lib\site-packages\lpips\weights\v0.1\alex.pth
LPIPS model loaded successfully.


In [5]:
#@title Prepare paths to evaluate images
IMGS_TEST = os.path.join(TEST_PATH, "Images_To_Test")
IMGS_GENE = os.path.join(TEST_PATH, "Images_Generated")
images_test = [os.path.join(IMGS_TEST, f) for f in os.listdir(IMGS_TEST) if f.endswith(".jpg")][:10]
print(f"Found {len(images_test)} test images.")
images_generated = [os.path.join(IMGS_GENE, f) for f in os.listdir(IMGS_GENE) if f.endswith(".png")]
print(f"Found {len(images_generated)} generated images.")

Found 10 test images.
Found 600 generated images.


In [ ]:
#@title Calculate LPIPS for each generated image against test images
for img_test in images_test:
    for img_gen in images_generated:
				# Get the model name from the generated image filename
				# For example, if the filename is sd-1.5-infarct-35662756.png, it should extract sd-1.5-infarct
        model_name = re.search(r'sd-\d\.\d-infarct(-lora)?', img_gen).group()
        lpips_score = calculate_lpips(img_gen, img_test)
        LPIPS_RESULTS.append((os.path.basename(img_test), os.path.basename(img_gen), model_name, lpips_score))
print("LPIPS calculated. Total:", len(LPIPS_RESULTS), "comparisons made.")

LPIPS calculated. Total: 6000


In [7]:
#@title Convert results to a DataFrame
data = np.array(LPIPS_RESULTS)
# Convertir el array a un DataFrame para un análisis más fácil
df = pd.DataFrame(data, columns=["id_test", "id_generada", "id_modelo", "score"])
df["score"] = df["score"].astype(float)  # Asegurar que los scores sean flotantes
#df
#@title Calculate average LPIPS score per model and test
df_avg = df.groupby(["id_test", "id_modelo"])["score"].mean().reset_index()
#df_avg

In [10]:
#@title Data pivot
pivot_df = df_avg.pivot(index="id_test", columns="id_modelo", values="score")
# Friendly column names
pivot_df = pivot_df.reset_index()
pivot_df.columns.name = None
pivot_df.round(4)

,id_test,sd-1.5-infarct,sd-1.5-infarct-lora,sd-2.1-infarct,sd-2.1-infarct-lora
0,TestInfarct (01).jpg,0.6941,0.6478,0.7236,0.6498
1,TestInfarct (02).jpg,0.6927,0.6486,0.7220,0.6471
2,TestInfarct (03).jpg,0.6866,0.6207,0.7108,0.6254
3,TestInfarct (04).jpg,0.7286,0.7109,0.7570,0.7249
4,TestInfarct (05).jpg,0.7111,0.6917,0.7392,0.7010
5,TestInfarct (06).jpg,0.7666,0.7730,0.7738,0.7818
6,TestInfarct (07).jpg,0.6991,0.6797,0.7166,0.7034
7,TestInfarct (08).jpg,0.6913,0.6689,0.7225,0.6679
8,TestInfarct (09).jpg,0.7254,0.7303,0.7555,0.7210
9,TestInfarct (10).jpg,0.7078,0.7067,0.7455,0.6969


In [9]:
#@title Results summary
print("Summary of LPIPS results:")
pivot_df.describe()

Summary of LPIPS results:


,sd-1.5-infarct,sd-1.5-infarct-lora,sd-2.1-infarct,sd-2.1-infarct-lora
count,10.000000,10.000000,10.000000,10.000000
mean,0.710317,0.687835,0.736659,0.691920
std,0.024415,0.044768,0.020794,0.045994
min,0.686592,0.620660,0.710799,0.625408
25%,0.693029,0.653685,0.722154,0.654290
50%,0.703405,0.685733,0.731399,0.698942
75%,0.721830,0.709840,0.753023,0.716605
max,0.766611,0.773039,0.773814,0.781810


## Make your own evaluations

In [16]:
#@title Imports and path base
from diffusers import DiffusionPipeline
MODELS_PATH = "models"
prompt = ("Elderly man at a sports stadium surrounded by a crowd, "
					"clutching his chest with a distressed look, indicating a heart attack."
				)
negative_prompt = (
					"blurry, deformed face, bad anatomy, poorly drawn face, out of focus, ugly, noisy, extra fingers, "
					"distorted, grainy, worst quality, low quality, low resolution, illustration, "
					"dull, watermark, close-up, 3d, 2d, painting, sketch, render, cartoon, grain, kitsch"
				)
trigger = "Person with expression of pain due to a heart attack, "
full_prompt = f"{trigger}, {prompt}"

In [ ]:
#@title Load the pre-trained models
sd15 = "stable-diffusion-v1-5/stable-diffusion-v1-5"
sd21 = "stabilityai/stable-diffusion-2-1-base"

sd15 = DiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, use_safetensors=True)
print("Loaded Stable Diffusion v1.5 model.")
sd21 = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base", torch_dtype=torch.float16, use_safetensors=True)
print("Loaded Stable Diffusion v2.1 model.")
# Move models to GPU if available
if torch.cuda.is_available():
		sd15.to("cuda")
		sd21.to("cuda")
		print("Models moved to GPU.")

Loading pipeline components...: 100%|██████████| 7/7 [00:13<00:00,  1.96s/it]


Loaded Stable Diffusion v1.5 model.


Loading pipeline components...: 100%|██████████| 6/6 [00:11<00:00,  1.99s/it]


Loaded Stable Diffusion v2.1 model.
Models moved to GPU.


In [18]:
#@title Evaluate base models with LPIPS
image1 = [os.path.join(IMGS_TEST, f) for f in os.listdir(IMGS_TEST) if f.endswith(".jpg")][0]

image_sd15 = sd15(prompt=full_prompt, negative_prompt=negative_prompt,
	guidance_scale=4, num_inference_steps=40).images[0]
image_sd21 = sd21(prompt=full_prompt, negative_prompt=negative_prompt,
	guidance_scale=4, num_inference_steps=40).images[0]

# Calculate distance LPIPS
lpips_score = calculate_lpips(image1, image_sd15)
print(f"LPIPS SD 1.5 Score: {lpips_score}")
lpips_score = calculate_lpips(image1, image_sd21)
print(f"LPIPS SD 2.1 Score: {lpips_score}")

100%|██████████| 40/40 [04:14<00:00,  6.37s/it]


LPIPS SD 1.5 Score: 0.7341530323028564
LPIPS SD 2.1 Score: 0.7580833435058594


In [ ]:
#@title Load LoRA tunning weights
sd15.load_lora_weights("models/sd-1.5-infarct-000010.safetensors")
print("Loaded LoRA weights for SD 1.5")
sd21.load_lora_weights("models/sd-2.1-infarct-000010.safetensors")
print("Loaded LoRA weights for SD 2.1")
# Move models to GPU if available
if torch.cuda.is_available():
	print("Moving models to GPU")
	sd15.to("cuda")
	sd21.to("cuda")

Loaded LoRA weights for SD 1.5
Loaded LoRA weights for SD 2.1
Moving models to GPU


In [20]:
#@title Evaluate LoRA models with LPIPS
image1 = [os.path.join(IMGS_TEST, f) for f in os.listdir(IMGS_TEST) if f.endswith(".jpg")][0]

image_sd15_lora = sd15(prompt=full_prompt, negative_prompt=negative_prompt,
	guidance_scale=4, num_inference_steps=40).images[0]
image_sd21_lora = sd21(prompt=full_prompt, negative_prompt=negative_prompt,
	guidance_scale=4, num_inference_steps=40).images[0]

# Calculate distance LPIPS
lpips_score = calculate_lpips(image1, image_sd15_lora)
print(f"LPIPS SD 1.5 with LoRA Score: {lpips_score}")
lpips_score = calculate_lpips(image1, image_sd21_lora)
print(f"LPIPS SD 2.1 with LoRA Score: {lpips_score}")

100%|██████████| 40/40 [05:06<00:00,  7.67s/it]


LPIPS SD 1.5 with LoRA Score: 0.6756401062011719
LPIPS SD 2.1 with LoRA Score: 0.6637046337127686
